# OSM, Nightlights and Remote Sensing Indicies
Now we add at every survey location features from OpenStreetMap, Nightlights (night-time luminosity) and remote sensing indicies from Sentinel. First off lets import the survey data to define the area of interest.

In [1]:
import sys
sys.path.append("../Src")

In [3]:
import pandas as pd
from utils import df_boundaries, points_to_polygon
data = pd.read_csv('../Data/datasets/sample_Moza.csv')
minlat, maxlat, minlon, maxlon = df_boundaries(data, buffer=0.05, lat_col="gpsLatitude", lon_col="gpsLongitude")
area = points_to_polygon(minlon, minlat, maxlon, maxlat)

### Nightlights
Nightlights are highly correlated with poverty (Neal, J.) and from our experience, to some extent with food security. For each AOI and relevant period, we derive the luminosity from the National Oceanic and Atmospheric Administration (NOAA) of the US Department of Commerce. In particular we use the NOAA Visible Infrared Imaging Radiometer Suite (VIIRS) monthly product, normalized by the DMSP-OLS Nighttime Lights Time Series, also from NOAA. The result is a georeferenced dataset with the luminosity shown on a 100 metre x 100 metre grid. Each surveyed cluster is then assigned an average luminosity from the area around its location. The core of the code that handles this data source is the nightlights class.

In [4]:
from nightlights import Nightlights

NGT = Nightlights(area, 
                  '../Data/Geofiles/nightlights/', 
                  '2018-01-01', 
                  '2018-08-01')

data['nightlights'] = NGT.nightlights_values(data)

INFO: downloading nightlights for area of interest ...


### OpenStreetMap
Open Street Map (OSM) is often the most accurate source of data on infrastructure in developing countries. All the data can be easily accessed and downloaded through the OverPass API.


We currently extract the location of schools and hospitals in the AOI from Open Street Map and compute the distance of the AOI to the closest school and hospital. Other infrastructure locations could be extracted such as parks, roads, and trees. However, the completeness of these layers varies a lot from country to country and even within the same country. We found that schools and hospitals have been the features most thoroughly mapped by OSM volunteers across our countries of interest.  

In [8]:
from osm import OSM_extractor
import numpy as np
        
OSM = OSM_extractor(minlon, minlat, maxlon, maxlat)
tags = {"amenity": ["school", "hospital"], "natural": ["tree"]}
osm_gdf = {}
osm_features = []

for key, values in tags.items():
    for value in values:
        osm_gdf["value"] = OSM.download(key, value)
        osm_tree = OSM.gpd_to_tree(osm_gdf["value"])
        dist = data.apply(OSM.distance_to_nearest, args=(osm_tree,), axis=1)
        data['distance_{}'.format(value)] = dist.apply(lambda x: np.log(0.0001 + x))
        osm_features.append('distance_{}'.format(value))        

INFO: Downloading OSM data for amenity = school
INFO: OSM data for amenity = school downloaded. N lines: 
INFO: Downloading OSM data for amenity = hospital
INFO: OSM data for amenity = hospital downloaded. N lines: 
INFO: Downloading OSM data for natural = tree
INFO: OSM data for natural = tree downloaded. N lines: 


### NDBI,NDVI,NDWI
Sentinel-2 is an Earth observation mission of the European Space Agency (ESA) launched in 2015. Multi-spectral images of the earth are available every five days at a resolution of ten metres. We use the Google Earth Engine platform to access and compute indices on these imageries. Currently, we extract three indices from four spectral bands (green, red, near infrared and shortwave infrared bands) at a given x, y location:


- NDVI: The Normalized Difference Vegetation Index is computed from the Red and Near-Infrared bands using the following formula : NDVI = (NIR - R) / (NIR + R). NDVI is used globally by the scientific community to monitor the health of vegetation
- NDWI: The Normalized Difference Water Index is computed from the Green and Infrared bands using the following formula: NDWI = (G - NIR) / (G + NIR)
- NDBI: The Normalized Difference Built-up Index is computed from the Panchromatic and Shortwave Infrared (SWIR) and Near-Infrared bands using the following formula: NDBI = (NIR - SWIR) / (NIR + SWIR)

In [9]:
area['coordinates']

[[[31.25342, -26.62219],
  [40.86516, -26.62219],
  [40.86516, -10.94615],
  [31.25342, -10.94615],
  [31.25342, -26.62219]]]

In [10]:
from rms_indexes import S2indexes

S2 = S2indexes(area, '../Data/Geofiles/NDs/', '2018-01-01', '2018-08-01', 'country')
S2.download()
data[['max_NDVI', 'max_NDBI', 'max_NDWI']] = S2.rms_values(data).apply(pd.Series)

INFO: downloading rms indexes for area of interest ...
INFO: NDs scope -> country
cb3d6c6fcd52d814a8fc7f99e6789fe1.NDVI_max.tif NDVI_max.tif
cb3d6c6fcd52d814a8fc7f99e6789fe1.NDBI_max.tif NDBI_max.tif
cb3d6c6fcd52d814a8fc7f99e6789fe1.NDWI_max.tif NDWI_max.tif


In [ ]:
data.to_csv("../Data/Features/features_demo_evaluation.csv", index=False)